In [ ]:
import requests
import pandas as pd
from datetime import datetime

In [ ]:
def crypto_data(id, days):
    info_endpoint = f"https://api.coingecko.com/api/v3/coins/{id}"
    info_response = requests.get(info_endpoint)

    if info_response.status_code == 200:
        coin_info = info_response.json()
        coin_name = coin_info["name"]
    else:
        print(f"Error al obtener la información de {id}. Detalles del error:")
        print(info_response.text)
        return None

    history_endpoint = f"https://api.coingecko.com/api/v3/coins/{id}/market_chart"
    history_params = {
        "vs_currency": "usd",
        "days": days,
        "interval": "daily"
    }

    history_response = requests.get(history_endpoint, params=history_params)

    if history_response.status_code == 200:
        history_data = history_response.json()
        timestamps = [entry[0] for entry in history_data["prices"]]
        prices = [entry[1] for entry in history_data["prices"]]
        market_caps = [entry[1] for entry in history_data["market_caps"]]
        total_volumes = [entry[1] for entry in history_data["total_volumes"]]

        dates = [datetime.fromtimestamp(ts / 1000).strftime('%Y-%m-%d') for ts in timestamps]

        df_historical = pd.DataFrame({
            "Date": dates,
            "Price": prices,
            "MarketCap": market_caps,
            "TotalVolume": total_volumes,
            "Currency": [coin_name] * len(timestamps)
        })

        return df_historical
    else:
        print(f"Error al obtener los datos históricos para {id}. Detalles del error:")
        print(history_response.text)
        return None

# Símbolos de las 10 monedas principales
top = ["bitcoin", "ethereum", "binancecoin", "ripple", "dogecoin"]

days = "14"

In [ ]:
df_min = pd.DataFrame()
for symbol in top:
    df_symbol = crypto_data(id=symbol, days=days)
    if df_symbol is not None:
        df_min = pd.concat([df_min, df_symbol], ignore_index=True)

df_min.to_csv("crypto_data.csv", index=False)

In [ ]:
df_min